# Practica de Visualización

Un dataset de NOAA está almacenado en el archivo `data/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. Este es el dataset que utilizaremos para la práctica.

Nota: Los datos para esta práctica forman parte de un dataset perteneciente a The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). El GHCN-Daily se compone de registros climáticos diarios de miles de estaciones de la superficie terrestre en todo el mundo.

Cada fila en el archivo de datos corresponde a una sola observación.

Nos proporciona las siguientes variables:

- **id** : código de identificación de la estación
- **date** : fecha en formato YYYY-MM-DD (por ejemplo: 2012-01-24 = 24 de enero de 2012)
- **element** : indica el tipo de medición
    - **TMAX** : Temperatura máxima
    - **TMIN** : Temperatura mínima
- **value**: valor de la temperatura (en décimas de grados centígrados)

## Indicaciones

1. Lee la documentación y familiarícese con el conjunto de datos, luego escriba un código python que arroje un gráfico lineal de las temperaturas máximas y mínimas récord por día del año durante el período 2005-2014. El área entre las temperaturas máximas y mínimas récord de cada día debe estar sombreada.
2. Superponga una dispersión de los datos de 2015 para cualquier punto (máximos y mínimos) para los cuales se superó el récord de diez años (2005-2014) máximo o mínimo histórico en 2015.
3. Debes tener cuidado con los días bisiestos (es decir, el 29 de febrero), es totalmente razonable eliminar estos puntos del conjunto de datos para esta visualización.
4. ¡Haz que la imagen sea agradable! Considera problemas como leyendas, etiquetas y líneas en demasía.

Los datos proporcionados están cerca de **Ann Arbor, Michigan, Estados Unidos**.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
# Armamos el dataframe y ya pasamos date a formato data time
df = pd.read_csv("../data/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv", parse_dates=['Date'])


In [ ]:
# Eliminamos del df los mes 2 y dia 29 para eliminar los 29 de febrero
df = df[~((df['Date'].dt.month == 2) & (df['Date'].dt.day == 29))]

In [ ]:
# Pasamos data value a grados celcius dividiendo entre 10
df['Data_Value'] = df['Data_Value'] / 10


In [ ]:
# Creamos columnas auxiliares una con año y otra con mes y dia
df['anio'] = df['Date'].dt.year
df['mes_dia'] = df['Date'].dt.strftime('%m-%d')


In [ ]:
# Filtramos el df en los años que nos interesan
df_hist = df[(df['anio'] >= 2005) & (df['anio'] <= 2014)].copy()
df_2015 = df[df['anio'] == 2015].copy()


In [ ]:
# Calculamos los records historicos por dia
tmax_hist = (df_hist[df_hist['Element'] == 'TMAX']
             .groupby('mes_dia')['Data_Value'].max())

tmin_hist = (df_hist[df_hist['Element'] == 'TMIN']
             .groupby('mes_dia')['Data_Value'].min())


In [ ]:
# Ahora lo mismo pero con 2015
tmax_2015 = (df_2015[df_2015['Element'] == 'TMAX'].groupby('mes_dia')['Data_Value'].max())

tmin_2015 = (df_2015[df_2015['Element'] == 'TMIN'].groupby('mes_dia')['Data_Value'].min())


In [ ]:
# Records rotos en 2015
record_max_2015 = tmax_2015[tmax_2015 > tmax_hist]
record_min_2015 = tmin_2015[tmin_2015 < tmin_hist]


In [ ]:
x_all = range(len(tmax_hist))
md_list = list(tmax_hist.index)

pos_high = [md_list.index(md) for md in record_max_2015.index]
pos_low = [md_list.index(md) for md in record_min_2015.index]

tmax_hist_c = tmax_hist.values / 10
tmin_hist_c = tmin_hist.values / 10
record_max_c = record_max_2015.values / 10
record_min_c = record_min_2015.values / 10

#Por seguridad, asegurar orden (mín <= máx) antes de sombrear
upper = np.maximum(tmax_hist_c, tmin_hist_c)
lower = np.minimum(tmax_hist_c, tmin_hist_c)


In [ ]:
#Gráfico
plt.figure(figsize=(13, 8))

#Sombreado entre mínimos y máximos históricos
plt.fill_between(x_all, upper, lower,
                 alpha=0.35, color='darkgrey',
                 label='Rango histórico 2005–2014')

#Línea de máximos históricos
plt.plot(x_all, tmax_hist_c, color='r', marker='+', markersize=4,
         label='Máximos históricos (2005–2014)')

#Línea de mínimos históricos
plt.plot(x_all, tmin_hist_c, color='b', marker='+', markersize=4,
         label='Mínimos históricos (2005–2014)')

#Puntos 2015 que baten récords
plt.scatter(pos_high, rec_max_c, marker='o', color="red", label='Nuevos máximos 2015')
plt.scatter(pos_low, rec_min_c, marker='o', color="blue", label='Nuevos mínimos 2015')

#Etiquetas y leyenda
plt.title('Récords diarios de temperatura cerca de Ann Arbor, MI (2005–2014 vs 2015)')
plt.xlabel('Días del año')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.grid(alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()